In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import data_utils
import train_utils
import models

%reload_ext autoreload
%autoreload 2

In [83]:
dev = data_utils.COAD_dataset(data_utils.COAD_DEV)
dev_loader = torch.utils.data.DataLoader(dev, batch_size=1, shuffle=True, pin_memory=True)
train = data_utils.COAD_dataset(data_utils.COAD_TRAIN)
train_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True, pin_memory=True)
valid = data_utils.COAD_dataset(data_utils.COAD_VALID)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=1, shuffle=False, pin_memory=True)

In [84]:
n_conv_layers = 2
kernel_size = [4,3]
n_conv_filters = [36,48]
hidden_size = 512
n_rnn_layers = 2
dropout=0.5
gen = models.Generator(n_conv_layers, kernel_size, n_conv_filters, hidden_size, n_rnn_layers, dropout=dropout)
gen.cuda()

n_conv_layers = 2
n_fc_layers = 2
kernel_size = [4,3]
n_conv_filters = [36,48]
hidden_size = [512,512]
dropout=0.5
enc = models.ConvNet(n_conv_layers, n_fc_layers, kernel_size, n_conv_filters, hidden_size, dropout=dropout)
enc.cuda()

lamb1 = 0
lamb2 = 0
xent = nn.CrossEntropyLoss()
learning_rate = 1e-4
temp = 10
params = list(enc.parameters()) + list(gen.parameters())
optimizer = torch.optim.Adam(params, lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, min_lr=1e-6)

In [81]:
def pool_fn(x):
    #v,a = torch.max(x,0)
    v = torch.mean(x,0)
    return v

In [85]:
for e in range(0,400):
    train_utils.rationales_training_loop_GS(e, train_loader, gen, enc, pool_fn, lamb1, lamb2, xent, learning_rate, optimizer,temp)
    if e > 30:
        lamb1 += 0.001
        temp -= 0.25
    temp = np.max([temp,1])
    lamb1 = np.min([lamb1,0.4])
    if e % 5 == 0:
        print('Lambda: {0:0.5f}, LR: {1:0.7f}, Temperature: {2:0.2f}'.format(lamb1, optimizer.state_dict()['param_groups'][0]['lr'],temp))
    train_utils.rationales_validation_loop_GS(e, valid_loader, gen, enc, pool_fn, xent, scheduler)

Epoch: 0, Train Loss: 34.627166, Train Omega: 0.0000, Fraction of Tiles: 0.4995
Lambda: 0.00000, LR: 0.0001000, Temperature: 10.00
Epoch: 0, Val Loss: 34.9664, Val Acc: 0.4082, Fraction of Tiles: 0.9999
Epoch: 1, Train Loss: 34.538411, Train Omega: 0.0000, Fraction of Tiles: 0.5022


/opt/anaconda/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch: 1, Val Loss: 0.0000, Val Acc: nan, Fraction of Tiles: 0.0000
Epoch: 2, Train Loss: 34.896113, Train Omega: 0.0000, Fraction of Tiles: 0.5012
Epoch: 2, Val Loss: 0.0000, Val Acc: nan, Fraction of Tiles: 0.0000
Epoch: 3, Train Loss: 34.326960, Train Omega: 0.0000, Fraction of Tiles: 0.4980
Epoch: 3, Val Loss: 0.0000, Val Acc: nan, Fraction of Tiles: 0.0000
Epoch: 4, Train Loss: 34.321365, Train Omega: 0.0000, Fraction of Tiles: 0.4990
Epoch: 4, Val Loss: 36.6588, Val Acc: 0.4082, Fraction of Tiles: 0.0094
Epoch: 5, Train Loss: 34.511874, Train Omega: 0.0000, Fraction of Tiles: 0.4969
Lambda: 0.00000, LR: 0.0001000, Temperature: 10.00
Epoch: 5, Val Loss: 0.0000, Val Acc: nan, Fraction of Tiles: 0.0000
Epoch: 6, Train Loss: 34.589235, Train Omega: 0.0000, Fraction of Tiles: 0.4991
Epoch: 6, Val Loss: 0.0000, Val Acc: nan, Fraction of Tiles: 0.0000
Epoch: 7, Train Loss: 34.622525, Train Omega: 0.0000, Fraction of Tiles: 0.4977
Epoch: 7, Val Loss: 0.0000, Val Acc: nan, Fraction of Til

Epoch: 52, Val Loss: 42.1780, Val Acc: 0.4082, Fraction of Tiles: 0.9999
Epoch: 53, Train Loss: 31.434392, Train Omega: 0.6816, Fraction of Tiles: 0.6019
Epoch: 53, Val Loss: 41.1668, Val Acc: 0.4082, Fraction of Tiles: 0.9999
Epoch: 54, Train Loss: 30.607916, Train Omega: 0.6478, Fraction of Tiles: 0.5088
Epoch: 54, Val Loss: 0.0000, Val Acc: nan, Fraction of Tiles: 0.0000
Epoch: 55, Train Loss: 36.449756, Train Omega: 0.0520, Fraction of Tiles: 0.0278
Lambda: 0.02500, LR: 0.0001000, Temperature: 3.75
Epoch: 55, Val Loss: 0.0000, Val Acc: nan, Fraction of Tiles: 0.0000
Epoch: 56, Train Loss: 34.606764, Train Omega: 0.4335, Fraction of Tiles: 0.2732
Epoch: 56, Val Loss: 45.8804, Val Acc: 0.4082, Fraction of Tiles: 0.9999
Epoch: 57, Train Loss: 33.662980, Train Omega: 0.8484, Fraction of Tiles: 0.6299
Epoch: 57, Val Loss: 35.5915, Val Acc: 0.4082, Fraction of Tiles: 0.9999
Epoch: 58, Train Loss: 31.378054, Train Omega: 0.8878, Fraction of Tiles: 0.6318
Epoch: 58, Val Loss: 36.7028, Val 

Epoch: 102, Val Loss: 21.2833, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 103, Train Loss: 26.805536, Train Omega: 3.4578, Fraction of Tiles: 0.9569
Epoch: 103, Val Loss: 21.2124, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 104, Train Loss: 26.378614, Train Omega: 3.5106, Fraction of Tiles: 0.9571
Epoch: 104, Val Loss: 21.2018, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 105, Train Loss: 26.308917, Train Omega: 3.5555, Fraction of Tiles: 0.9580
Lambda: 0.07500, LR: 0.0000010, Temperature: 1.00
Epoch: 105, Val Loss: 21.2190, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 106, Train Loss: 26.661258, Train Omega: 3.6032, Fraction of Tiles: 0.9570
Epoch: 106, Val Loss: 21.1620, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 107, Train Loss: 26.216940, Train Omega: 3.6458, Fraction of Tiles: 0.9552
Epoch: 107, Val Loss: 21.1588, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 108, Train Loss: 26.460515, Train Omega: 3.7015, Fraction of Tiles: 0.9573
Epoch: 108, Va

Epoch: 152, Train Loss: 26.908113, Train Omega: 5.8057, Fraction of Tiles: 0.9545
Epoch: 152, Val Loss: 20.3695, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 153, Train Loss: 26.055824, Train Omega: 5.8498, Fraction of Tiles: 0.9522
Epoch: 153, Val Loss: 20.3392, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 154, Train Loss: 25.941678, Train Omega: 5.8929, Fraction of Tiles: 0.9537
Epoch: 154, Val Loss: 20.3071, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 155, Train Loss: 26.332943, Train Omega: 5.9416, Fraction of Tiles: 0.9528
Lambda: 0.12500, LR: 0.0000010, Temperature: 1.00
Epoch: 155, Val Loss: 20.2683, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 156, Train Loss: 26.365450, Train Omega: 5.9846, Fraction of Tiles: 0.9530
Epoch: 156, Val Loss: 20.2682, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 157, Train Loss: 25.963399, Train Omega: 6.0366, Fraction of Tiles: 0.9523
Epoch: 157, Val Loss: 20.2649, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 158, Tr

Epoch: 201, Val Loss: 19.5903, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 202, Train Loss: 25.734923, Train Omega: 8.0614, Fraction of Tiles: 0.9343
Epoch: 202, Val Loss: 19.5911, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 203, Train Loss: 25.283201, Train Omega: 8.0929, Fraction of Tiles: 0.9327
Epoch: 203, Val Loss: 19.5771, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 204, Train Loss: 25.884117, Train Omega: 8.1002, Fraction of Tiles: 0.9271
Epoch: 204, Val Loss: 19.5051, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 205, Train Loss: 26.437945, Train Omega: 8.1859, Fraction of Tiles: 0.9321
Lambda: 0.17500, LR: 0.0000010, Temperature: 1.00
Epoch: 205, Val Loss: 19.5734, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 206, Train Loss: 26.447261, Train Omega: 8.2267, Fraction of Tiles: 0.9331
Epoch: 206, Val Loss: 19.5932, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 207, Train Loss: 25.928892, Train Omega: 8.2909, Fraction of Tiles: 0.9341
Epoch: 207, Va

Epoch: 266, Train Loss: 25.677051, Train Omega: 10.6645, Fraction of Tiles: 0.8906
Epoch: 266, Val Loss: 18.8416, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 267, Train Loss: 25.475914, Train Omega: 10.6703, Fraction of Tiles: 0.8902
Epoch: 267, Val Loss: 18.9027, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 268, Train Loss: 26.452384, Train Omega: 10.7136, Fraction of Tiles: 0.8892
Epoch: 268, Val Loss: 18.9189, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 269, Train Loss: 24.589580, Train Omega: 10.7598, Fraction of Tiles: 0.8905
Epoch: 269, Val Loss: 18.8951, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 270, Train Loss: 24.506346, Train Omega: 10.8257, Fraction of Tiles: 0.8899
Lambda: 0.24000, LR: 0.0000010, Temperature: 1.00
Epoch: 270, Val Loss: 18.8920, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 271, Train Loss: 25.306681, Train Omega: 10.8531, Fraction of Tiles: 0.8890
Epoch: 271, Val Loss: 18.8862, Val Acc: 0.8776, Fraction of Tiles: 0.9999
Epoch: 2

Epoch: 315, Val Loss: 18.5903, Val Acc: 0.9184, Fraction of Tiles: 0.9999
Epoch: 316, Train Loss: 25.619804, Train Omega: 11.9814, Fraction of Tiles: 0.8194
Epoch: 316, Val Loss: 18.5914, Val Acc: 0.9184, Fraction of Tiles: 0.9999
Epoch: 317, Train Loss: 25.519359, Train Omega: 11.9755, Fraction of Tiles: 0.8112
Epoch: 317, Val Loss: 18.7382, Val Acc: 0.9184, Fraction of Tiles: 0.9730
Epoch: 318, Train Loss: 25.032951, Train Omega: 12.0887, Fraction of Tiles: 0.8176
Epoch: 318, Val Loss: 18.5971, Val Acc: 0.9184, Fraction of Tiles: 0.9999
Epoch: 319, Train Loss: 25.661654, Train Omega: 12.2259, Fraction of Tiles: 0.8178
Epoch: 319, Val Loss: 18.6294, Val Acc: 0.9184, Fraction of Tiles: 0.9999
Epoch: 320, Train Loss: 25.773186, Train Omega: 12.3531, Fraction of Tiles: 0.8205
Lambda: 0.29000, LR: 0.0000010, Temperature: 1.00
Epoch: 320, Val Loss: 18.6760, Val Acc: 0.9184, Fraction of Tiles: 0.9999
Epoch: 321, Train Loss: 24.815492, Train Omega: 12.3116, Fraction of Tiles: 0.8164
Epoch: 3

Epoch: 365, Val Loss: 14.4508, Val Acc: 0.9388, Fraction of Tiles: 0.0753
Epoch: 366, Train Loss: 29.388672, Train Omega: 3.0711, Fraction of Tiles: 0.0651
Epoch: 366, Val Loss: 14.5758, Val Acc: 0.9388, Fraction of Tiles: 0.0733
Epoch: 367, Train Loss: 29.693639, Train Omega: 3.0049, Fraction of Tiles: 0.0641
Epoch: 367, Val Loss: 14.6971, Val Acc: 0.9388, Fraction of Tiles: 0.0740
Epoch: 368, Train Loss: 29.472274, Train Omega: 3.0435, Fraction of Tiles: 0.0630
Epoch: 368, Val Loss: 14.5840, Val Acc: 0.9388, Fraction of Tiles: 0.0734
Epoch: 369, Train Loss: 30.397076, Train Omega: 2.5187, Fraction of Tiles: 0.0292
Epoch: 369, Val Loss: 15.0384, Val Acc: 0.8723, Fraction of Tiles: 0.0338
Epoch: 370, Train Loss: 30.577954, Train Omega: 2.3201, Fraction of Tiles: 0.0222
Lambda: 0.34000, LR: 0.0000010, Temperature: 1.00
Epoch: 370, Val Loss: 14.9983, Val Acc: 0.8723, Fraction of Tiles: 0.0218
Epoch: 371, Train Loss: 30.714497, Train Omega: 2.3254, Fraction of Tiles: 0.0227
Epoch: 371, Va

In [88]:
train_utils.rationales_validation_loop_GS(e, train_loader, gen, enc, pool_fn, xent, scheduler)

Epoch: 399, Val Loss: 16.3430, Val Acc: 0.8049, Fraction of Tiles: 0.0228
